# Dataloading 01

In this notebook, we'll figure out how to use PyTorch's DataLoader class to load our massive files without reading the entirety of them into memory

In [3]:
import comet_ml
import dask.dataframe as dd
import pandas as pd 
import torch
import linecache 
import csv
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn.functional as F
import sys, os
from pathlib import Path
import plotly.express as px 
from sklearn.utils.class_weight import compute_class_weight
import torch

sys.path.append('../src')
sys.path.append('../src/models/lib')
here = Path().cwd()

We'll first design a custom dataset to use with PyTorch's `DataLoader` class

In [5]:
from models.lib.neural import *

In [6]:
primary = GeneExpressionData(
    filename='../data/processed/primary.csv',
    labelname='../data/processed/meta_primary_labels.csv',
    class_label='Subtype'
)

In [8]:
class Model(nn.Module):
    def __init__(self, inputs, outputs):
        super(Model, self).__init__()

        layers = [
            nn.Linear(1024, 1024),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.BatchNorm1d(1024, 1024),
        ]

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(inputs, 1024),
            *layers,
            nn.Linear(1024, outputs),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = Model(
    inputs = primary.num_features(),
    outputs = primary.num_labels(),
)

In [19]:
train_size = int(0.80 * len(primary))
test_size = len(primary) - train_size

train, test = torch.utils.data.random_split(primary, [train_size, test_size])
trainloader = DataLoader(
    train, 
    batch_size=2,
)

validloader = DataLoader(
    test, 
    batch_size=2, 
)

In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [25]:
epochs = 5
min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    model.train()     # Optional when not using Model Specific layer
    for data, labels in trainloader:
        optimizer.zero_grad()
        target = model(data)
        loss = criterion(target,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    valid_loss = 0.0
    model.eval() # Optional when not using Model Specific layer
    for data, labels in validloader:
        target = model(data)
        loss = criterion(target,labels)
        valid_loss = loss.item() * data.size(0)

    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(trainloader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

tensor(3.3497, grad_fn=<NllLossBackward0>)
tensor(3.4142, grad_fn=<NllLossBackward0>)
tensor(3.3813, grad_fn=<NllLossBackward0>)
tensor(3.3865, grad_fn=<NllLossBackward0>)
tensor(3.4073, grad_fn=<NllLossBackward0>)
tensor(3.3748, grad_fn=<NllLossBackward0>)
tensor(3.3703, grad_fn=<NllLossBackward0>)
tensor(3.4311, grad_fn=<NllLossBackward0>)
tensor(3.3937, grad_fn=<NllLossBackward0>)
tensor(3.3895, grad_fn=<NllLossBackward0>)
tensor(3.3704, grad_fn=<NllLossBackward0>)
tensor(3.3707, grad_fn=<NllLossBackward0>)
tensor(3.4057, grad_fn=<NllLossBackward0>)
tensor(3.3983, grad_fn=<NllLossBackward0>)
tensor(3.3866, grad_fn=<NllLossBackward0>)
tensor(3.3999, grad_fn=<NllLossBackward0>)
tensor(3.4105, grad_fn=<NllLossBackward0>)
tensor(3.3919, grad_fn=<NllLossBackward0>)
tensor(3.3900, grad_fn=<NllLossBackward0>)
tensor(3.3581, grad_fn=<NllLossBackward0>)
tensor(3.3828, grad_fn=<NllLossBackward0>)
tensor(3.3731, grad_fn=<NllLossBackward0>)
tensor(3.3738, grad_fn=<NllLossBackward0>)
tensor(3.39

tensor(3.3541, grad_fn=<NllLossBackward0>)
tensor(3.4143, grad_fn=<NllLossBackward0>)
tensor(3.4109, grad_fn=<NllLossBackward0>)
tensor(3.3859, grad_fn=<NllLossBackward0>)
tensor(3.3671, grad_fn=<NllLossBackward0>)
tensor(3.3136, grad_fn=<NllLossBackward0>)
tensor(3.3513, grad_fn=<NllLossBackward0>)
tensor(3.3585, grad_fn=<NllLossBackward0>)
tensor(3.3640, grad_fn=<NllLossBackward0>)
tensor(3.3557, grad_fn=<NllLossBackward0>)
tensor(3.3499, grad_fn=<NllLossBackward0>)
tensor(3.3748, grad_fn=<NllLossBackward0>)
tensor(3.3777, grad_fn=<NllLossBackward0>)
tensor(3.3579, grad_fn=<NllLossBackward0>)
tensor(3.3933, grad_fn=<NllLossBackward0>)
tensor(3.3531, grad_fn=<NllLossBackward0>)
tensor(3.3352, grad_fn=<NllLossBackward0>)
tensor(3.3612, grad_fn=<NllLossBackward0>)
tensor(3.3883, grad_fn=<NllLossBackward0>)
tensor(3.3565, grad_fn=<NllLossBackward0>)
tensor(3.3556, grad_fn=<NllLossBackward0>)
tensor(3.3393, grad_fn=<NllLossBackward0>)
tensor(3.4001, grad_fn=<NllLossBackward0>)
tensor(3.39

tensor(3.3493, grad_fn=<NllLossBackward0>)
tensor(3.3040, grad_fn=<NllLossBackward0>)
tensor(3.3192, grad_fn=<NllLossBackward0>)
tensor(3.3744, grad_fn=<NllLossBackward0>)
tensor(3.3718, grad_fn=<NllLossBackward0>)
tensor(3.3504, grad_fn=<NllLossBackward0>)
tensor(3.3348, grad_fn=<NllLossBackward0>)
tensor(3.3419, grad_fn=<NllLossBackward0>)
tensor(3.3102, grad_fn=<NllLossBackward0>)
tensor(3.3990, grad_fn=<NllLossBackward0>)
tensor(3.3984, grad_fn=<NllLossBackward0>)
tensor(3.3017, grad_fn=<NllLossBackward0>)
tensor(3.3512, grad_fn=<NllLossBackward0>)
tensor(3.3917, grad_fn=<NllLossBackward0>)
tensor(3.3378, grad_fn=<NllLossBackward0>)
tensor(3.3010, grad_fn=<NllLossBackward0>)
tensor(3.3826, grad_fn=<NllLossBackward0>)
tensor(3.3303, grad_fn=<NllLossBackward0>)
tensor(3.3993, grad_fn=<NllLossBackward0>)
tensor(3.3419, grad_fn=<NllLossBackward0>)
tensor(3.3955, grad_fn=<NllLossBackward0>)
tensor(3.3770, grad_fn=<NllLossBackward0>)
tensor(3.3952, grad_fn=<NllLossBackward0>)
tensor(3.33

tensor(3.3945, grad_fn=<NllLossBackward0>)
tensor(3.2781, grad_fn=<NllLossBackward0>)
tensor(3.4013, grad_fn=<NllLossBackward0>)
tensor(3.3872, grad_fn=<NllLossBackward0>)
tensor(3.3891, grad_fn=<NllLossBackward0>)
tensor(3.3787, grad_fn=<NllLossBackward0>)
tensor(3.3559, grad_fn=<NllLossBackward0>)
tensor(3.3117, grad_fn=<NllLossBackward0>)
tensor(3.3114, grad_fn=<NllLossBackward0>)
tensor(3.4067, grad_fn=<NllLossBackward0>)
tensor(3.3396, grad_fn=<NllLossBackward0>)
tensor(3.4316, grad_fn=<NllLossBackward0>)
tensor(3.3444, grad_fn=<NllLossBackward0>)
tensor(3.3273, grad_fn=<NllLossBackward0>)
tensor(3.3514, grad_fn=<NllLossBackward0>)
tensor(3.3722, grad_fn=<NllLossBackward0>)
tensor(3.3954, grad_fn=<NllLossBackward0>)
tensor(3.3790, grad_fn=<NllLossBackward0>)
tensor(3.2757, grad_fn=<NllLossBackward0>)
tensor(3.3900, grad_fn=<NllLossBackward0>)
tensor(3.2428, grad_fn=<NllLossBackward0>)
tensor(3.3161, grad_fn=<NllLossBackward0>)
tensor(3.3403, grad_fn=<NllLossBackward0>)
tensor(3.40

tensor(3.3477, grad_fn=<NllLossBackward0>)
tensor(3.2958, grad_fn=<NllLossBackward0>)
tensor(3.3789, grad_fn=<NllLossBackward0>)
tensor(3.3174, grad_fn=<NllLossBackward0>)
tensor(3.3697, grad_fn=<NllLossBackward0>)
tensor(3.2638, grad_fn=<NllLossBackward0>)
tensor(3.3427, grad_fn=<NllLossBackward0>)
tensor(3.3468, grad_fn=<NllLossBackward0>)
tensor(3.4099, grad_fn=<NllLossBackward0>)
tensor(3.3309, grad_fn=<NllLossBackward0>)
tensor(3.3572, grad_fn=<NllLossBackward0>)
tensor(3.3260, grad_fn=<NllLossBackward0>)
tensor(3.3087, grad_fn=<NllLossBackward0>)
tensor(3.3166, grad_fn=<NllLossBackward0>)
tensor(3.3692, grad_fn=<NllLossBackward0>)
tensor(3.3991, grad_fn=<NllLossBackward0>)
tensor(3.2896, grad_fn=<NllLossBackward0>)
tensor(3.3009, grad_fn=<NllLossBackward0>)
tensor(3.3515, grad_fn=<NllLossBackward0>)
tensor(3.3835, grad_fn=<NllLossBackward0>)
tensor(3.3855, grad_fn=<NllLossBackward0>)
tensor(3.4067, grad_fn=<NllLossBackward0>)
tensor(3.3769, grad_fn=<NllLossBackward0>)
tensor(3.32

...

tensor(2.4868, grad_fn=<NllLossBackward0>)
tensor(2.8122, grad_fn=<NllLossBackward0>)
tensor(2.6223, grad_fn=<NllLossBackward0>)
tensor(3.7826, grad_fn=<NllLossBackward0>)
tensor(3.1016, grad_fn=<NllLossBackward0>)
tensor(2.0000, grad_fn=<NllLossBackward0>)
tensor(3.3886, grad_fn=<NllLossBackward0>)
tensor(3.1843, grad_fn=<NllLossBackward0>)
tensor(2.8094, grad_fn=<NllLossBackward0>)
tensor(2.3865, grad_fn=<NllLossBackward0>)
tensor(4.1151, grad_fn=<NllLossBackward0>)
tensor(1.7296, grad_fn=<NllLossBackward0>)
tensor(3.2151, grad_fn=<NllLossBackward0>)
tensor(2.4436, grad_fn=<NllLossBackward0>)
tensor(3.3461, grad_fn=<NllLossBackward0>)
tensor(2.6731, grad_fn=<NllLossBackward0>)
tensor(3.7212, grad_fn=<NllLossBackward0>)
tensor(2.5487, grad_fn=<NllLossBackward0>)
tensor(4.0366, grad_fn=<NllLossBackward0>)
tensor(4.0263, grad_fn=<NllLossBackward0>)
tensor(2.0714, grad_fn=<NllLossBackward0>)
tensor(3.5986, grad_fn=<NllLossBackward0>)
tensor(2.9955, grad_fn=<NllLossBackward0>)
tensor(3.65

tensor(2.6504, grad_fn=<NllLossBackward0>)
tensor(2.8725, grad_fn=<NllLossBackward0>)
tensor(1.7243, grad_fn=<NllLossBackward0>)
tensor(2.4910, grad_fn=<NllLossBackward0>)
tensor(1.7214, grad_fn=<NllLossBackward0>)
tensor(3.0557, grad_fn=<NllLossBackward0>)
tensor(2.7110, grad_fn=<NllLossBackward0>)
tensor(2.6664, grad_fn=<NllLossBackward0>)
tensor(3.8168, grad_fn=<NllLossBackward0>)
tensor(2.7279, grad_fn=<NllLossBackward0>)
tensor(2.6654, grad_fn=<NllLossBackward0>)
tensor(2.8689, grad_fn=<NllLossBackward0>)
tensor(3.5433, grad_fn=<NllLossBackward0>)
tensor(4.6764, grad_fn=<NllLossBackward0>)
tensor(2.7376, grad_fn=<NllLossBackward0>)
tensor(2.3351, grad_fn=<NllLossBackward0>)
tensor(3.0311, grad_fn=<NllLossBackward0>)
tensor(3.3009, grad_fn=<NllLossBackward0>)
tensor(2.2221, grad_fn=<NllLossBackward0>)
tensor(2.9995, grad_fn=<NllLossBackward0>)
tensor(2.8636, grad_fn=<NllLossBackward0>)
tensor(2.9881, grad_fn=<NllLossBackward0>)
tensor(2.0578, grad_fn=<NllLossBackward0>)
tensor(1.72

tensor(3.4622, grad_fn=<NllLossBackward0>)
tensor(3.0402, grad_fn=<NllLossBackward0>)
tensor(3.5751, grad_fn=<NllLossBackward0>)
tensor(4.3968, grad_fn=<NllLossBackward0>)
tensor(2.9854, grad_fn=<NllLossBackward0>)
tensor(2.3451, grad_fn=<NllLossBackward0>)
tensor(2.0035, grad_fn=<NllLossBackward0>)
tensor(3.2261, grad_fn=<NllLossBackward0>)
tensor(3.1507, grad_fn=<NllLossBackward0>)
tensor(3.7020, grad_fn=<NllLossBackward0>)
tensor(3.1758, grad_fn=<NllLossBackward0>)
tensor(3.5792, grad_fn=<NllLossBackward0>)
tensor(1.7500, grad_fn=<NllLossBackward0>)
tensor(3.0362, grad_fn=<NllLossBackward0>)
tensor(2.9820, grad_fn=<NllLossBackward0>)
tensor(3.8668, grad_fn=<NllLossBackward0>)
tensor(3.1173, grad_fn=<NllLossBackward0>)
tensor(3.2068, grad_fn=<NllLossBackward0>)
tensor(3.6561, grad_fn=<NllLossBackward0>)
tensor(2.3278, grad_fn=<NllLossBackward0>)
tensor(3.6182, grad_fn=<NllLossBackward0>)
tensor(3.3821, grad_fn=<NllLossBackward0>)
tensor(3.3016, grad_fn=<NllLossBackward0>)
tensor(2.89

tensor(3.1576, grad_fn=<NllLossBackward0>)
tensor(3.7675, grad_fn=<NllLossBackward0>)
tensor(1.7646, grad_fn=<NllLossBackward0>)
tensor(2.8196, grad_fn=<NllLossBackward0>)
tensor(2.9645, grad_fn=<NllLossBackward0>)
tensor(2.6190, grad_fn=<NllLossBackward0>)
tensor(3.5350, grad_fn=<NllLossBackward0>)
tensor(2.6629, grad_fn=<NllLossBackward0>)
tensor(2.3464, grad_fn=<NllLossBackward0>)
tensor(2.6951, grad_fn=<NllLossBackward0>)
tensor(3.5580, grad_fn=<NllLossBackward0>)
tensor(3.4748, grad_fn=<NllLossBackward0>)
tensor(2.8833, grad_fn=<NllLossBackward0>)
tensor(2.3530, grad_fn=<NllLossBackward0>)
tensor(2.2294, grad_fn=<NllLossBackward0>)
tensor(2.3915, grad_fn=<NllLossBackward0>)
tensor(3.1085, grad_fn=<NllLossBackward0>)
tensor(3.8102, grad_fn=<NllLossBackward0>)
tensor(2.3479, grad_fn=<NllLossBackward0>)
tensor(3.6093, grad_fn=<NllLossBackward0>)
tensor(2.5483, grad_fn=<NllLossBackward0>)
tensor(3.5352, grad_fn=<NllLossBackward0>)
tensor(2.3439, grad_fn=<NllLossBackward0>)
tensor(2.73

KeyboardInterrupt: 